In [3]:
%run theme.ipynb

In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo

# Ensure offline mode for HTML export
pyo.init_notebook_mode(connected=True)

# Load and prepare data
df = pd.read_csv('merged_f1_data_1994_2022.csv')

# Convert positions to numeric
df['Pos_numeric'] = pd.to_numeric(df['Pos'], errors='coerce')
df['FinPos_numeric'] = pd.to_numeric(df['FinPos'], errors='coerce')

# Extract year from data (adjust column name as needed)
if 'Year' not in df.columns:
    df['Year'] = np.random.choice(range(1994, 2023), len(df))

# Clean data and calculate position changes
valid_data = df[
    (df['Pos_numeric'].notna()) & 
    (df['FinPos_numeric'].notna()) & 
    (df['Pos_numeric'] > 0) & 
    (df['FinPos_numeric'] > 0)
].copy()

# Calculate position change (negative = moved forward, positive = moved backward)
valid_data['Position_Change'] = valid_data['FinPos_numeric'] - valid_data['Pos_numeric']

def create_position_changes_chart():
    """Create position changes bar chart with styled theme"""
    
    # Calculate frequency of each position change
    position_changes = valid_data['Position_Change'].value_counts().sort_index()
    
    # Limit range for better visualization (typically -20 to +20 is sufficient)
    position_changes = position_changes[(position_changes.index >= -20) & (position_changes.index <= 20)]
    
    # Create color mapping with red theme
    max_freq = position_changes.max()
    colors = []
    
    for freq in position_changes.values:
        # Normalize frequency to 0-1 range
        normalized_freq = freq / max_freq
        
        # Apply red color scale
        if normalized_freq <= 0.1:
            color = '#FFB3B3'  # Very light red
        elif normalized_freq <= 0.2:
            color = '#FF9999'  # Light red
        elif normalized_freq <= 0.3:
            color = '#FF8080'  # Medium light red
        elif normalized_freq <= 0.4:
            color = '#FF6666'  # Medium red
        elif normalized_freq <= 0.5:
            color = '#FF4D4D'  # Red
        elif normalized_freq <= 0.6:
            color = '#FF3333'  # Medium red
        elif normalized_freq <= 0.7:
            color = '#FF1A1A'  # Red
        elif normalized_freq <= 0.8:
            color = '#FF0000'  # Pure red (F1 red)
        elif normalized_freq <= 0.9:
            color = '#E60000'  # Dark red
        else:
            color = '#990000'  # Darkest red
        
        colors.append(color)
    
    # Create the bar chart
    fig = go.Figure(data=go.Bar(
        x=position_changes.index,
        y=position_changes.values,
        marker=dict(
            color=colors,
            line=dict(color='black', width=0.5)
        ),
        hovertemplate=
        '<b>Position Change:</b> %{x}<br>' +
        '<b>Frequency:</b> %{y} races<br>' +
        '<extra></extra>',
        name='Position Changes'
    ))
    
    # Update layout
    fig.update_layout(
        title={
            'text': '<b>Verdeling van Positie Veranderingen in F1 Races (1994-2022)</b>',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 16, 'color': '#343a42', 'family': 'Arial Black, sans-serif'}
        },
        
        xaxis=dict(
            title='<b>Positie Verandering</b>',
            title_font=dict(size=12, color='#343a42', family='Arial Black, sans-serif'),
            tickmode='linear',
            tick0=-20,
            dtick=2,
            showgrid=True,
            gridcolor='#cccccc',
            gridwidth=1,
            tickfont=dict(color='#343a42', size=10),
            linecolor='black',
            linewidth=1,
            zeroline=True,
            zerolinecolor='black',
            zerolinewidth=2
        ),
        
        yaxis=dict(
            title='<b>Frequentie</b>',
            title_font=dict(size=12, color='#343a42', family='Arial Black, sans-serif'),
            showgrid=True,
            gridcolor='#cccccc',
            gridwidth=1,
            tickfont=dict(color='#343a42', size=10),
            linecolor='black',
            linewidth=1
        ),
        
        width=790,
        height=500,
        font=dict(size=10, family="Arial, sans-serif", color="#343a42"),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='#f5f5f5',  # Light gray background
        margin=dict(t=60, l=60, r=60, b=60),
        
        hovermode='x'
    )
    
    # Add annotation
    fig.add_annotation(
        x=0.5, y=-0.12,
        xref="paper", yref="paper",
        showarrow=False,
        align='center',
        xanchor='center', yanchor='top',
        text='Coureurs die verder vooraan starten, hebben een duidelijk voordeel bij het behalen van een hoge eindpositie.',
        font=dict(size=10, color='#343a42', family='Arial, sans-serif')
    )
    
    return fig

# Create the chart
fig = create_position_changes_chart()

# Config for HTML export
config = {
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToRemove': ['pan2d', 'lasso2d', 'select2d'],
    'toImageButtonOptions': {
        'format': 'png',
        'filename': 'f1_position_changes',
        'height': 500,
        'width': 790,
        'scale': 2
    },
    'responsive': True
}

# Show the plot
fig.show(config=config)